# Ejercicio: *Predicción del clima usando una red neuronal* 
**Objetivo**

Entrenar una red neuronal que pueda predecir el estado del clima basándose en la
temperatura y la fecha.

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#Generar datos aleatorios
np.random.seed(69)
n_samples = 1000
#Generar fechas aleatorias en 2024
dates = pd.date_range(start="2024-01-01", end="2024-12-31", periods=n_samples)
#Generar temperaturas aleatorias (entre -10°C y 40°C)
temperatures = np.random.uniform(-10, 40, n_samples)
#Generar estados del clima en función de la temperatura
#Soleado: temp > 25, Nublado: 10 <= temp <= 25, Lluvioso: temp < 10
conditions = np.where(temperatures > 29, "soleado",
                      np.where(temperatures >= 11, "nublado", "lluvioso"))
#Humedad: desde el 0% hasta el 100% 
stats = np.random.randint(20, 101, n_samples)
#Crear el DataFrame
data = pd.DataFrame({
    "fecha": dates,
    "temperatura": temperatures,
    "clima": conditions,
    "humedad": stats
})
#Convertir la columna 'fecha' en valores numéricos (para simplificar el modelo)
data['dia'] = data['fecha'].dt.dayofyear  # Día del año (1 a 365)
data.drop(columns="fecha", inplace=True)  # Eliminar la columna original de fechas
print(data.head())
#Codificar el clima (soleado=0, nublado=1, lluvioso=2)
encoder = LabelEncoder()
data['clima_encoded'] = encoder.fit_transform(data['clima'])
#Variables de entrada (día, temperatura, humedad) y salida (clima codificado)
X = data[['dia', 'temperatura', 'humedad']].values
y = data['clima_encoded'].values
#Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#Escalar los datos de entrada
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Crear el modelo
model = Sequential([
    Dense(64, activation='relu', input_shape=(3,)),  # Entrada ahora con tres características
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # Salida: 3 clases (soleado, nublado, lluvioso)
])

#Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#Entrenar el modelo
model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))
#Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Pérdida: {loss:.2f}, Precisión: {accuracy:.2%}")
#Realizar predicciones con humedad
new_data = np.array([
    [120, 15, 75],  # Día 120 del año, temperatura 15°C, humedad 75%
    [200, 30, 40],  # Día 200, temperatura 30°C, humedad 40%
    [300, -5, 85]])  # Día 300, temperatura -5°C, humedad 85%
#Escalar las nuevas entradas
new_data_scaled = scaler.transform(new_data)
#Realizar predicciones
predictions = model.predict(new_data_scaled)
#Convertir predicciones a etiquetas
predicted_classes = np.argmax(predictions, axis=1)
predicted_labels = encoder.inverse_transform(predicted_classes)
#Mostrar los resultados
for i, pred in enumerate(predicted_labels):
    print(f"Entrada: Día {new_data[i][0]}, Temp {new_data[i][1]}°C, Humedad {new_data[i][2]}% => Clima predicho: {pred}")


   temperatura     clima  humedad  dia
0     4.812458  lluvioso       52    1
1    30.453386   soleado       22    1
2     7.512626  lluvioso       88    1
3    29.470463   soleado       23    2
4    18.067449   nublado       51    2
Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4816 - loss: 1.0010 - val_accuracy: 0.8450 - val_loss: 0.8426
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8604 - loss: 0.8006 - val_accuracy: 0.8600 - val_loss: 0.6430
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8742 - loss: 0.6247 - val_accuracy: 0.8750 - val_loss: 0.4661
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8993 - loss: 0.4540 - val_accuracy: 0.8850 - val_loss: 0.3481
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9243 - loss: 0.3548 - val_accuracy: 0.8950 - val_loss: 0.2787
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9217 - loss: 0.2947 - val_accuracy: 0.9300 - val_loss: 0.2357
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9280 - loss: 0.2541 - val_accuracy: 0.9300 - val_loss: 0.2048
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9299 - loss: 0.2230 - val_accuracy: 0.9450 - val_loss: 0.1781
Ep

# Explicacion

con los datos generados como 

fecha : en las fecha generamos los datos y los convertimos a valores numericos para que sean identificables para la inteligencia artificial, es un valor de entrada

temperatura : para la temperatura usando un rango de temperatura entre -10 y 40 para obtener un valor mejorado del modelo y posteriormente con

clima : agregamos los climas y los casos en los que damos alternativas si la temperatura esta mayor que 25, "soleado y si esta menor o igual que 10 puede ser "nublado" o "lluvioso")

humedad  : por ultimo agregamos el porcentaje de humedad puede tener el dataframe para luego agregar al codigo como una variable de entrada para darle mas precision